### PYCARET AUTOML FOR CUSTOMER CHURN PREDICTION

In [7]:
!pip install seaborn
!pip install catboost


print("Libraries installated Sucessfully!")

   ---------------------------------------- 0.0/102.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/102.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/102.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/102.5 MB ? eta -:--:--
   ---------------------------------------- 0.5/102.5 MB 699.0 kB/s eta 0:02:26
   ---------------------------------------- 0.5/102.5 MB 699.0 kB/s eta 0:02:26
   ---------------------------------------- 0.8/102.5 MB 714.3 kB/s eta 0:02:23
   ---------------------------------------- 0.8/102.5 MB 714.3 kB/s eta 0:02:23
   ---------------------------------------- 1.0/102.5 MB 751.1 kB/s eta 0:02:16
   ---------------------------------------- 1.0/102.5 MB 751.1 kB/s eta 0:02:16
    --------------------------------------- 1.3/102.5 MB 713.8 kB/s eta 0:02:22
    --------------------------------------- 1.3/102.5 MB 713.8 kB/s eta 0:02:22
    --------------------------------------- 1.6/102.5 MB 635.7 kB/s eta 0:02

In [8]:
# import necessary libraries

import pandas as pd
import numpy as np
from pycaret.classification import *
import warnings
warnings.filterwarnings('ignore')


from sklearn.metrics import precision_recall_curve, roc_curve, fbeta_score
from sklearn.calibration import calibration_curve
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

print("Imports Done!")

Imports Done!


In [9]:
# pycaret AutoML for customer churn

print("\n ======================LOADING PREPROCESSED DATA======================")

# Load the data you prepared earlier
churn_predictive_data = pd.read_csv('data/churn_predictive_data.csv')
# make copy before manipulations
churn_data = churn_predictive_data.copy()


print(f"\nData loaded: {churn_data.shape[0]:,} customers, {churn_data.shape[1]} features")
display(churn_data.head())


 ======================LOADING PREPROCESSED DATA======================

Data loaded: 10,000 customers, 11 features


,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### PYCARET SETUP WITH IMPACTFUL PARAMETERS

### PyCaret's AutoML:

- Finds the best model and hyperparameters

- Handles preprocessing and feature engineering

- Does cross-validation and model comparison

In [10]:

print("\n =====================PYCARET SETUP WITH ALL USEFUL PARAMETERS=====================")

setup_params = {
    'data': churn_data,                   # Your data
    'target': 'exited',                   # Target column
    'train_size': 0.8,                    # 80% for training
    'test_data': None,                    # Will be created automatically
    'preprocess': True,                   # Let PyCaret handle preprocessing
    'imputation_type': 'simple',          # Simple imputation for any missing
    'numeric_imputation': 'median',       # Fill numeric with median
    'categorical_imputation': 'mode',     # Fill categorical with mode
    
    # critical for our imbalance of (3.9:1 ratio)
    'fix_imbalance': True,                # Handle class imbalance
    'fix_imbalance_method': 'smote',      # Use SMOTE for balancing
    
    # Feature engineering
    'remove_multicollinearity': True,     # Remove highly correlated features
    'multicollinearity_threshold': 0.9,   # Threshold for correlation
    'remove_outliers': True,              # Remove outliers using PCA
    'outliers_threshold': 0.05,           # Remove top 5% outliers
    'feature_selection': True,            # Perform feature selection
    'feature_selection_method': 'classic',        # Classic feature selection
    'feature_selection_estimator': 'lightgbm',    # Use LightGBM for selection
    
    # Encoding
    'categorical_features': ['geography', 'gender'], # Specify categorical
   
    
    # Scaling
    'normalize': True,                    # Normalize features
    'normalize_method': 'zscore',         # Use z-score normalization
    
    # Transformation
    'transformation': True,               # Make data more Gaussian-like
    'transformation_method': 'yeo-johnson', # Yeo-Johnson transformation
    
    # Ignore features
    'ignore_features': [],                # No features to ignore
    
    # Session
    'session_id': 42,                     # For reproducibility
    'verbose': True,                      # Detailed output
    'profile': False,                     # Don't create data profile (can be heavy)
}

# Initialize setup
print("\nInitializing PyCaret setup...")
# setup_params.pop("high_cardinality_features", None)
# setup_params.pop("high_cardinality_method", None)
 
clf_setup = setup(**setup_params)

# Display setup information
print("\n===============================SETUP COMPLETE===============================")
print(f"   - Dataset Shape: {clf_setup.dataset.shape}")
print(f"   - Features Processed: {clf_setup.X_train.shape[1]}")


print(
    f"   - Target Distribution: "
    f"{clf_setup.y_train.value_counts().to_dict()}"
)



 =====================PYCARET SETUP WITH ALL USEFUL PARAMETERS=====================

Initializing PyCaret setup...


,Description,Value
0,Session id,42
1,Target,exited
2,Target type,Binary
3,Original data shape,"(10000, 11)"
4,Transformed data shape,"(14110, 3)"
5,Transformed train set shape,"(12110, 3)"
6,Transformed test set shape,"(2000, 3)"
7,Ordinal features,1
8,Numeric features,8
9,Categorical features,2



===============================SETUP COMPLETE===============================
   - Dataset Shape: (10000, 11)
   - Features Processed: 10
   - Target Distribution: {0: 6370, 1: 1630}


### COMPARE MODELS WITH IMBALANCE-FOCUSED METRICS

In [22]:
# define metrics that matter for imbalanced data

sort_by_metric = 'F1'  # F1-score balances precision and recall

# F1: balances both false positives and false negatives.
# Other good options: 'Recall', 'AUC', 'Precision'

print(f"Comparing models sorted by: '{sort_by_metric}'")
# For imbalanced data, F1 and AUC are most important

print("For imbalanced data, F1 and AUC are most important")

# Compare all models
print("Training and comparing models...")

# Only include models that handle imbalance well
include_models = ['lr', 'ridge', 'lda', 'gbc', 'ada', 'lightgbm', 'qda']

# Compare selected models
print("Training and comparing models optimized for imbalance...")
best_model = compare_models(
    include=include_models,     # Only these models
    fold=10,                    # 10-fold cross-validation
    round=4,                    # Round metrics to 4 decimals
    sort=sort_by_metric,        # Sort by chosen metric
    verbose=True,               # Show progress
    cross_validation=True,
)

Comparing models sorted by: 'F1'
For imbalanced data, F1 and AUC are most important
Training and comparing models...
Training and comparing models optimized for imbalance...


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.5173,0.5047,0.4923,0.2090,0.2934,0.0105,0.0129,0.5850
lr,Logistic Regression,0.5217,0.5009,0.4790,0.2072,0.2888,0.0072,0.0093,0.4390
ridge,Ridge Classifier,0.5189,0.0000,0.4797,0.2065,0.2885,0.0056,0.0069,0.3500
qda,Quadratic Discriminant Analysis,0.5560,0.5059,0.4110,0.1982,0.2060,0.0052,0.0002,0.4490
lightgbm,Light Gradient Boosting Machine,0.7927,0.4896,0.0091,0.2750,0.0176,0.0036,0.0126,0.8170
gbc,Gradient Boosting Classifier,0.7959,0.4828,0.0014,0.1500,0.0028,0.0008,0.0049,2.6660
ada,Ada Boost Classifier,0.7963,0.5025,0.0000,0.0000,0.0000,0.0000,0.0000,1.1890


### lr: Logistic Regression. This is the best model
* Selected from prioritizing `F1`
* `F1`: Best for imbalanced data and it balances the precion and recall
* `F1`: balances both false positives and false negatives.

### CREATE LOGISTIC REGRESSION MODEL WITH HYPERPARAMETERS

In [5]:
### Creating the best Model
print("\n ===============================CREATING LOGISTIC REGRESSION MODEL===============================")

# Create Logistic Regression with specific parameters for imbalance
lr_model = create_model(
    'lr',    
    fold=10,
    cross_validation=True,
    class_weight='balanced',   # VERY important for minority churn
    verbose=True
)


 ===============================CREATING LOGISTIC REGRESSION MODEL===============================


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.4538,0.5702,0.7423,0.2345,0.3564,0.0677,0.1029
1,0.5025,0.5764,0.7485,0.2547,0.3801,0.1092,0.1545
2,0.4762,0.5947,0.8037,0.2529,0.3847,0.1083,0.1654
3,0.7062,0.7663,0.7301,0.3839,0.5032,0.3221,0.3557
4,0.4562,0.5898,0.7669,0.2395,0.3650,0.0789,0.1215
5,0.6762,0.7375,0.7239,0.3554,0.4768,0.2800,0.3172
6,0.4688,0.5767,0.7546,0.2421,0.3666,0.0840,0.1257
7,0.4550,0.5478,0.6933,0.2265,0.3414,0.0494,0.0726
8,0.4462,0.5732,0.7485,0.2328,0.3552,0.0644,0.0995


### From the above

* Recall is consistently high => Mean = 74.85%
This is good for churn prediction, because the model catches ~75% of `actual churners across folds`.
* Precision is low, Mean = 26.75% Means many predicted churners are actually not churning (high false positives).
* For BankCo: This could mean retention offers are sent to many loyal customers, `higher cost`.
### Pupose of the Bank

* If BankCo wants to save as many churners as possible, Thi model (Recall = 74%) is okay despite higher cost.

### Model Analysis

`Strengths for churn prediction`:

- High recall => catches most churners.

- Better than random guessing (AUC > 0.5).

### Weaknesses:

* Low precision => high false positive rate => costly campaigns.

* Low precision is problematic => Model may be too expensive.

* Model instability across folds => unreliable in production.

* Overall discriminative power is weak (AUC only 0.61).



<!-- ### CALIBRATE PROBABILITIES, `IMPORTANT FOR IMBALANCE` -->

In [30]:
# ADD NEW CELL BEFORE finalize_model()

print("\n" + "="*80)
print("PROBABILITY CALIBRATION FOR BETTER CHURN PREDICTIONS")
print("="*80)

print("Calibrating probabilities for more reliable churn predictions...")
calibrated_model = calibrate_model(
    lr_model,
    method='isotonic',  # Better for imbalanced data than 'sigmoid'
    fold=10,
    verbose=True
)

# update lr_model to calibrated version
# lr_model = calibrated_model

# verify calibration
print("\n calibration makes Probabilities are now more reliable for now")
print("A probability of 0.40 means ~40% of similar customers churn")


PROBABILITY CALIBRATION FOR BETTER CHURN PREDICTIONS
Calibrating probabilities for more reliable churn predictions...


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.4762,0.5718,0.7178,0.2388,0.3583,0.0757,0.1093
1,0.5012,0.5756,0.7485,0.2542,0.3795,0.1082,0.1533
2,0.5012,0.5941,0.7239,0.2500,0.3717,0.0986,0.1377
3,0.7175,0.7727,0.7055,0.3925,0.5044,0.3286,0.3562
4,0.5050,0.6003,0.6933,0.2462,0.3633,0.0896,0.1222
5,0.6425,0.7420,0.7669,0.3351,0.4664,0.2552,0.3048
6,0.4938,0.5788,0.7178,0.2458,0.3662,0.0899,0.1265
7,0.4538,0.5507,0.6933,0.2260,0.3409,0.0484,0.0713
8,0.4588,0.5714,0.7485,0.2374,0.3604,0.0739,0.1118



 calibration makes Probabilities are now more reliable for now
A probability of 0.40 means ~40% of similar customers churn


### `After calibration , calibrate_model`:
* When the model outputs `0.40`, it means that about 40 out of 100 similar customers are expected to churn, making the predicted probabilities `reliable` for decisions.


### FINALIZE ON ENTIRE DATASET

In [31]:

print("\n =====================FINALIZING MODEL=====================")

print("finalizing model on entire dataset...")
finalized_model = finalize_model(lr_model)

print("=> Model finalized")
print("   - Trained on 100% of available data")
print("   - and Ready for deployment")


 =====================FINALIZING MODEL=====================
finalizing model on entire dataset...
=> Model finalized
   - Trained on 100% of available data
   - and Ready for deployment


###  PROBABILITY THRESHOLD TUNING

In [25]:
df  = pd.read_csv("data/churn_train_pycaret.csv")
test_data = df.copy()
test_data.head()

,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,753,France,Male,57,7,0.00,1,1,0,159475.08,1
1,739,Germany,Male,32,3,102128.27,1,1,0,63981.37,1
2,755,Germany,Female,37,0,113865.23,2,1,1,117396.25,0
3,561,France,Male,37,5,0.00,2,1,0,83093.25,0
4,692,Germany,Male,49,6,110540.43,2,0,1,107472.99,0


In [34]:
# Get predictions
predictions = predict_model(finalized_model, data=test_data)
predictions.head(50)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5222,0.5869,0.6669,0.2490,0.3626,0.0936,0.1231


,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,churn_probability,stay_probability,churn_confirmed,risk_category,exited,prediction_label,prediction_score
0,753,France,Male,57,7,0.000000,1,1,0,159475.078125,0.6109,0.3891,1,High Risk,1,0,0.6109
1,739,Germany,Male,32,3,102128.273438,1,1,0,63981.371094,0.5574,0.4426,0,High Risk,1,1,0.5574
2,755,Germany,Female,37,0,113865.226562,2,1,1,117396.250000,0.5717,0.4283,0,High Risk,0,1,0.5717
3,561,France,Male,37,5,0.000000,2,1,0,83093.250000,0.6275,0.3725,1,High Risk,0,0,0.6275
4,692,Germany,Male,49,6,110540.429688,2,0,1,107472.992188,0.5717,0.4283,0,High Risk,0,1,0.5717
5,659,France,Female,38,9,0.000000,2,1,1,132809.187500,0.6129,0.3871,1,High Risk,0,0,0.6129
6,782,France,Male,34,9,0.000000,1,1,0,183021.062500,0.5805,0.4195,0,High Risk,1,0,0.5805
7,546,France,Male,24,8,156325.375000,1,1,1,125381.023438,0.5756,0.4244,0,High Risk,0,1,0.5756
8,756,Germany,Male,41,2,124439.492188,2,0,1,47093.109375,0.5668,0.4332,0,High Risk,0,1,0.5668
9,685,Spain,Male,34,6,83264.281250,1,0,0,9663.280273,0.5187,0.4813,0,High Risk,0,0,0.5187


In [36]:
# # Get predictions
# predictions = predict_model(finalized_model, data=test_data)

# # Find probability column
# probability_column = None
# for col in predictions.columns:
#     if col.lower() in ['score', 'prediction_score', 'probability', 'prob']:
#         probability_column = col
#         break
# if probability_column is None:
#     float_cols = predictions.select_dtypes(include=['float64', 'float32']).columns
#     probability_column = float_cols[-1] if len(float_cols) > 0 else predictions.columns[-1]

# # Add probabilities
# test_data['churn_probability'] = predictions[probability_column]
# test_data['stay_probability'] = 1 - test_data['churn_probability']

# # Imbalance-aware threshold (0.35 for 80:20 data)
# # threshold = 0.35
# threshold = 0.6
# test_data['churn_confirmed'] = (test_data['churn_probability'] >= threshold).astype(int)

# # Add risk categories
# def get_risk_category(prob):
#     if prob >= 0.5:
#         return 'High Risk'
#     elif prob >= 0.35:
#         return 'Medium Risk'
#     elif prob >= 0.25:
#         return 'Low Risk'
#     elif prob >= 0.15:
#         return 'Watch'
#     else:
#         return 'Safe'

# test_data['risk_category'] = test_data['churn_probability'].apply(get_risk_category)

# # Save results
# test_data.to_csv('data/churn_predictions_final.csv', index=False)

# # Show summary
# print(f"Customers analyzed: {len(test_data):,}")
# print(f"Flagged for churn (threshold={threshold}): {test_data['churn_confirmed'].sum():,}")

# # Display top at-risk customers
# # top_risks = test_data[['churn_probability', 'risk_category', 'churn_confirmed']]
# # top_risks = top_risks.sort_values('churn_probability', ascending=False).head(10)
# # display(top_risks)
# test_data.tail(50)

### Plot confusion matrix

In [35]:
plot_model(
    finalized_model,
    plot='confusion_matrix',
    plot_kwargs={'percent': True},
    save=True
)

'Confusion Matrix.png'

In [ ]:



print("AUC-ROC Curve:")
plot_model(
    finalized_model,
    plot='auc',
    save=True
)



# 7.3 Precision-Recall Curve, BEST for imbalance
print(" Precision-Recall Curve (recommended for imbalance):")
plot_model(
    finalized_model,
    plot='pr',
    save=True
)

# 7.4 Feature Importance
print(" Feature Importance:")
try:
    plot_model(
        finalized_model,
        plot='feature',
        save=True
    )
except:
    print(" Feature importance plot not available for ensemble")
    # Plot learning curve instead
    plot_model(
        finalized_model,
        plot='learning',
        save=True
    )

### THRESHOLD ANALYSIS FOR IMBALANCED DATA


### `For 3.9:1 imbalance ratio, we can consider these thresholds`: 
- data is imbalanced (here 3.9 non-churners for every 1 churner)
- 0.5 (default): Balanced but misses churners
- 0.3-0.4: Better recall, catches more churners
- 0.25: High recall, many false positives

---
Problem for weak churn prediction:  80% of available data show customer who didn't churn, while only 20% are those who churned. And I'm interested to predict the customer who might leave, the minority!!

How can I approach this problem of imbalanced data to ensure I can have the best model that can predict minority!?
The problem is that: A model can get 80% accuracy just by predicting “everyone stayed” but that’s useless for churn.

---


### SAVE EVERYTHING

In [ ]:

# Save model

save_model(finalized_model, 'saved_model/churn_lr_final')
print("Model saved: 'churn_lr_final.pkl'")